# WARNING: OUTDATED!
This uses the old API. To be updated.

In [2]:
import casadi as cs
from urdf2casadi import casadi_geom as csg

In [3]:
# urdf2casadi uses cs.MX and cs.SX, which can be hard to read as these are sparse matrices.
# This short function just makes it so that the result will be a numpy matrix
def cs2np(asd):
    return cs.Function("temp",[],[asd])()["o0"].toarray()
# NOTE: casadi imports numpy as np, so cs.np is numpy

# Available functions:

In [4]:
for funcname in dir(csg):
    if funcname != "cs" and funcname[0] != "_": # Only show non-casadi and non-private functions
        print funcname

T_full_symbolic
T_prismatic
T_revolute
dual_quaternion_axis_rotation
dual_quaternion_axis_translation
dual_quaternion_conj
dual_quaternion_inv
dual_quaternion_norm2
dual_quaternion_prismatic
dual_quaternion_product
dual_quaternion_revolute
dual_quaternion_rpy
dual_quaternion_to_transformation_matrix
dual_quaternion_translation
quaternion_product
quaternion_revolute


# Transformation matrices

We have homogeneous 4x4 transformation matrices to represent the reference frames attached to the robot.
These are defined in the "translate then rotate" approach. This is because it is the most common approach. Where we use roll-pitch-yaw to define a rotation matrix, we follow the ZYX convention for the Euler angles.
Example of a simple roll-pitch-yaw and x-y-z displacement frame is:

In [28]:
roll = 0.0
pitch = 0.0
yaw = 0.0
x = 0.
y = 1.
z = 1.
T_example = csg.T_full_symbolic([x,y,z], [roll,pitch,yaw])
print type(T_example)
print "\ncasadi format:\n",T_example
print "\nReadable format:\n",cs2np(T_example)

<class 'casadi.casadi.SX'>

casadi format:
@1=1, @2=0, 
[[@1, @2, @2, @2], 
 [@2, @1, @2, @1], 
 [@2, @2, @1, @1], 
 [@2, @2, @2, @1]]

Readable format:
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  1.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  0.  1.]]


Change rpy and xyz if you'd like. The rotation matrix and displacements are easily extracted by:

In [33]:
print "Rotation matrix"
print cs2np(T_example[:3, :3])
print "Displacement"
print cs2np(T_example[:3, 3])

Rotation matrix
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
Displacement
[[ 0.]
 [ 1.]
 [ 1.]]


And of course, we can multiply matrices by:

In [35]:
res = cs.mtimes(T_example,T_example)
print cs2np(res)

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  2.]
 [ 0.  0.  1.  2.]
 [ 0.  0.  0.  1.]]


# Robot transformation matrices

Robots are usually equipped with prismatic and revolute joints. So we have transformation matrices T_prismatic and T_revolute.

## Prismatic joints

All joint transformation matrices follow the description in URDFs. Basically the joint origin relative the parent frame is described by a displacement, and then a RPY rotation. The axis is the vector we move along, defined in the joint origin frame. Example:

In [37]:
xyz = [0.,
       0.,
       0.]
rpy = [0.,
       0.,
       0.]
axis = [1., 
        0., 
        0.]
joint_value = 5.
T_p = csg.T_prismatic(xyz, rpy, axis, joint_value)
print cs2np(T_p)

[[ 1.  0.  0.  5.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


Try different axes and joint_values

## Revolute joints


Just as the prismatic joint we first get the displacement and rotation to the joint origin frame. Then we have an axis and a joint value that defines the rotation. The revolute joint follows an axis-angle rotation. However, axis-angle uses unit vectors for the axis, so make sure that it is a unit vector.
If you comment out the axis normalization, you can see that the determinant is no longer equal to 1, and our rotation matrix must be malformed. (A determinant of 1 is a requirement of rotation matrices on rigid transformations.)

In [57]:
xyz = [0.,
       0.,
       0.]
rpy = [0.,
       0.,
       0.]
axis = [0.1, 
        0., 
        0.]
axis_normal = cs.sqrt(axis[0]*axis[0]+axis[1]*axis[1]+axis[2]*axis[2])
axis = [element/axis_normal for element in axis] # Normalizing the axis
joint_value = 1.
T_r = csg.T_revolute(xyz, rpy, axis, joint_value)
print cs2np(T_r)
print "determinant(Rotation)=",cs.det(T_r[:3,:3])

[[ 1.          0.          0.          0.        ]
 [ 0.          0.54030231 -0.84147098  0.        ]
 [ 0.          0.84147098  0.54030231  0.        ]
 [ 0.          0.          0.          1.        ]]
determinant(Rotation)= 1


The rest is advanced stuff

# Quaternion and Dual Quaternion

## Quaternion

A quaternion is a representation of a rotation. We define
$$
\vec{q} = w + xi + yj + zk = \begin{bmatrix}
x\\
y\\
z\\
w
\end{bmatrix}
$$
where $i,j,k$ are the quaternion units, and the vector form is how we usually think of it. Rotations are defined using unit quaternions for which:
$$
\lvert\lvert q \lvert\lvert = \sqrt{w^2+x^2+y^2+z^2} = 1
$$
and conjugates defined as $q^* = w - xi -yj - zk$. With unit quaternions we have that the inverse of a quaternion is the conjugate.

They are related to the axis ($\vec{k}$) angle ($\theta$) parameters by:
$$
q = \begin{bmatrix}
\vec{k}\sin(\theta)\\
\cos(\theta)
\end{bmatrix} = \begin{bmatrix}
\vec{\epsilon}\\
\eta
\end{bmatrix}
$$
where $\eta,\vec{\epsilon}$ are the Euler parameters from the axis-angle definition. 
Addition of quaternions in vector form is just like normal vectors, but multiplication uses the quaternion product:
$$
q_1\otimes q_2 = \begin{bmatrix}
\eta_1\vec{\epsilon}_2 + \eta_2\vec{\epsilon}_1 + \vec{\epsilon}_1\times \vec{\epsilon}_2\\
\eta_1\eta_2 -\vec{\epsilon}_1^T\vec{\epsilon}_2
\end{bmatrix}=
\begin{bmatrix}
w_1 x_2 + x_1 w_2 + y_1 z_2 - z_1 y_2\\
w_1 y_2 - x_1 z_2 + y_1 w_2 + z_1 x_2\\
w_1 z_2 + x_1 y_2 - y_1 x_2 + z_1 w_2\\
w_1 w_2 - x_1 x_2 - y_1 y_2 - z_1 z_2
\end{bmatrix}
$$
The identity unit quaternion is:
$$
q_{id} = \begin{bmatrix}
0\\
0\\
0\\
1
\end{bmatrix}
$$
with $q_{id}\otimes q_1 = q_1$

In [62]:
q_id = [0,0,0,1]
q_1 = [1., 2., 0.23134, 12312.0]
print cs2np(csg.quaternion_product(q_id,q_1))

[[  1.00000000e+00]
 [  2.00000000e+00]
 [  2.31340000e-01]
 [  1.23120000e+04]]


Now since the quaternions are so obviously tied to Euler parameters, conversion from unit quaternions to rotation matrices is pretty much just shoving the values into the equation for converting Euler parameters to rotation matrices:
$$
\mathbf{R}(q) = \begin{bmatrix}
w^2 + x^2 - y^2 - z^2 & 2(x y - w z) & 2(x z + w y)\\
2(x y + w z) & w^2 - x^2 + y^2 - z^2 & 2(y z - w x)\\
2(x z - w y) & 2(y z - w x) & w^2 - x^2 - y^2 + z^2
\end{bmatrix}
$$
And just as matrix multiplication is used to transform a rotation to another, given a quaternion $p_0$ describing an orientation, the final orientation $p_f$ after a rotation $q$ is:
$$
p_f = q\otimes p_0 \otimes q^*
$$

We include the revolute joint form of a quaternion. The usage is exactly the same as with the revolute joint transformation matrix. And if you want a bit more visualization of the quaternions to try to get a bit better intuition around it. We suggest the 3blue1brown videos on quaternions:
[3Blue1Brown - What are quaternions, and how do you visualize them? A study of four dimensions.](https://www.youtube.com/watch?v=d4EgbgTm0Bg)
But that's not what's the most interesting here. Because we have dual quaternions!

## Dual quaternions

Just as quaternions represent rotations, dual quaternions represent transformation matrices!
A dual quaternion is a somewhat strange concept, but here are the basics:
A dual quaternion $Q$ is defined by a "real" quaternion, $q_r$, and a "dual" quaternion $q_d$ such that:
$$
Q = q_r + \varepsilon q_d = \begin{bmatrix}
x_r\\
y_r\\
z_r\\
w_r\\
x_d\\
y_d\\
z_d\\
w_d
\end{bmatrix} = \begin{bmatrix}
q_r\\
q_d
\end{bmatrix}
$$
where $\varepsilon$ is the dual unit with the magical property that $\varepsilon^2=0$, and the vector form is our own internal representation. So what does the dual quaternion product look like? Suppose we just apply the quaternion product with the magical $\varepsilon^2=0$ property and the knowledge that the quaternion product is distributative?
$$
Q_1\otimes Q_2 = (q_{r1} + \varepsilon q_{d1})\times(q_{r2} + \varepsilon q_{d2}) = q_{r1}\otimes q_{r2} + \varepsilon(q_{r1}\otimes q_{d2} + q_{d1}\otimes q_{r2})
$$
which has a real part and a dual part. Thus we have the dual quaternion product.

Now suppose $q_r$ is a unit quaternion that represents a rotation $R$. We note that the dual quaternion product will always just multiply the rotation part with rotations, just like with the transformation matrix, and that the rotations affect the summing of displacements. We know how to convert a quaternion to a rotation matrix, how about a $q_d$ to a displacement?

The definition usually used is that a translation $\vec{p} = [\Delta_x, \Delta_y, \Delta_z, 0]$ (quaternion form) is written in the dual portion of a dual quaternion by:
$$
q_d = \frac{1}{2}\vec{p}\otimes q_r
$$
this stems from the whole "translation THEN rotation" thing, if you switch that around, this switches too.
Let's play with it!

In [18]:
# Translation
axis_t = [0.1, 
          0.1,
          0.1]
translation_val = 10.
# Rotation
axis_r = [0.,
        0.,
        1.]
angle = 0.0
axis_normal = cs.sqrt(axis[0]*axis[0] + axis[1]*axis[1] + axis[2]*axis[2])
axis = [element/axis_normal for element in axis]
Q_t = csg.dual_quaternion_axis_translation(axis_t, translation_val)
Q_r = csg.dual_quaternion_axis_rotation(axis, angle)
Q_f = csg.dual_quaternion_product(Q_t,Q_r)
print "Dual Quaternion:\n",cs2np(Q_f)
print "Transformation Matrix:\n", cs2np(csg.dual_quaternion_to_transformation_matrix(Q_f))

Dual Quaternion:
[[ 0. ]
 [ 0. ]
 [ 0. ]
 [ 1. ]
 [ 0.5]
 [ 0.5]
 [ 0.5]
 [ 0. ]]
Transformation Matrix:
[[ 1.  0.  0.  1.]
 [ 0.  1.  0.  1.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  0.  1.]]


## Prismatic joints

In [19]:
xyz = [0., 
       0.,
       1.]
rpy = [0.,
       0.,
       0.]
axis_t = [0.1, 
          0.1,
          0.1]
joint_val = 10.
Q_p = csg.dual_quaternion_prismatic(xyz,
                                    rpy,
                                    axis_t,
                                    joint_val)
print "Prismatic Joint Dual Quaternion:\n", cs2np(Q_p)
print "Transformation matrix:\n", cs2np(csg.dual_quaternion_to_transformation_matrix(Q_p))

Prismatic Joint Dual Quaternion:
[[ 0. ]
 [ 0. ]
 [ 0. ]
 [ 1. ]
 [ 0.5]
 [ 0.5]
 [ 1. ]
 [ 0. ]]
Transformation matrix:
[[ 1.  0.  0.  1.]
 [ 0.  1.  0.  1.]
 [ 0.  0.  1.  2.]
 [ 0.  0.  0.  1.]]


## Revolute joint

In [22]:
xyz = [0., 
       0.,
       1.]
rpy = [0.,
       0.,
       0.]
axis_r = [1., 
          0.,
          0.]
axis_normal = cs.sqrt(axis_r[0]*axis_r[0] + axis_r[1]*axis_r[1] + axis_r[2]*axis_r[2])
axis_r = [element/axis_normal for element in axis_r]
joint_val = 10.
Q_p = csg.dual_quaternion_prismatic(xyz,
                                    rpy,
                                    axis_r,
                                    joint_val)
print "Revolute Joint Dual Quaternion:\n", cs2np(Q_p)
print "Transformation matrix:\n", cs2np(csg.dual_quaternion_to_transformation_matrix(Q_p))

Revolute Joint Dual Quaternion:
[[ 0. ]
 [ 0. ]
 [ 0. ]
 [ 1. ]
 [ 5. ]
 [ 0. ]
 [ 0.5]
 [ 0. ]]
Transformation matrix:
[[  1.   0.   0.  10.]
 [  0.   1.   0.   0.]
 [  0.   0.   1.   1.]
 [  0.   0.   0.   1.]]
